In [1]:
import sys
import psycopg2 as pg2  # Preferred cursor connection
from sqlalchemy import create_engine  # preferred for pushing back to DB
import yaml
import pandas as pd
import numpy as np

In [2]:
# Might need your own path...
with open('/data/users/dschnelb/secrets.yaml', 'r') as f:
        # loads contents of secrets.yaml into a python dictionary
        secret_config = yaml.safe_load(f.read())

# Set database connection to `conn`
db_params = secret_config['db']
conn = pg2.connect(host=db_params['host'],
                    port=db_params['port'],
                    dbname=db_params['dbname'],
                    user=db_params['user'],
                    password=db_params['password'])

# Connect cursor with psycopg2 database connection
cur = conn.cursor()

# Raw data for all_snapshots, student at grade 10 and above

## use DataFrame `raw`

In [3]:
qry = '''
SELECT * from clean.all_snapshots
where grade >= 10;
'''

cur.execute(qry)

rows = cur.fetchall()

# Build dataframe from rows
raw = pd.DataFrame(rows, columns=[name[0] for name in cur.description])

# Make sure student_id is an int
raw['student_lookup'] = raw['student_lookup'].astype('int')

raw.head()

,student_lookup,district,school_code,grade,school_year,birth_date,city,days_absent,days_absent_excused,days_absent_unexcused,...,special_ed,state,street,street2,withdraw_reason,withdrawn_to_irn,zip,status,street_clean,id
0,46,TriValley,TVHS,12,2014,1996-12-01,dresden,81.5,7.0,74.5,...,None,OH,11955 Bottom Rd,None,did not withdraw,None,43821-9423,active,11955 Bottom Rd,2557
1,538,Zanesville,None,10,2011,1995-04-01,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,open_enrollment,None,4094
2,558,Zanesville,None,11,2013,1996-09-01,None,29.0,NaN,NaN,...,None,None,None,None,None,None,None,open_enrollment,None,161648
3,584,Zanesville,None,11,2011,1993-05-01,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,open_enrollment,None,4287
4,629,Zanesville,None,11,2011,1993-02-01,None,NaN,NaN,NaN,...,None,None,None,None,None,None,None,open_enrollment,None,4479


In [4]:
raw = raw.replace([None],np.nan)

In [19]:
all_students = raw[(raw['grade']==12) & (~raw['school_year'].isin([2006, 2007, 2008]))].groupby(['district','school_year']).agg({'student_lookup':'count'})

In [20]:
withdraw_na = raw[(raw['withdraw_reason'].isna()) & (raw['grade']==12) & (~raw['school_year'].isin([2006, 2007, 2008]))]
withdraw_na

,student_lookup,district,school_code,grade,school_year,birth_date,city,days_absent,days_absent_excused,days_absent_unexcused,...,special_ed,state,street,street2,withdraw_reason,withdrawn_to_irn,zip,status,street_clean,id
12,4699,Zanesville,NaN,12,2015,1997-12-01,zanesville,NaN,NaN,NaN,...,NaN,OH,834 Race Street,NaN,NaN,NaN,43701,0,834 Race Street,164407
14,6594,Morgan,NaN,12,2015,1998-01-01,stockport,NaN,NaN,NaN,...,NaN,OH,5090 Tabor Ridge Road,NaN,NaN,NaN,43787,7,5090 Tabor Ridge Rd,201936
16,6727,Morgan,NaN,12,2015,1997-12-01,stockport,17.1,NaN,NaN,...,NaN,OH,PO Box 463,NaN,NaN,NaN,43787,7,Po Box 463,242179
20,7924,Zanesville,NaN,12,2011,1992-04-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,open_enrollment,NaN,34315
38,14724,Zanesville,NaN,12,2011,1993-04-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,open_enrollment,NaN,60630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59755,701169,New Lexington City SD,5,12,2011,1994-04-01,new lexington,0.0,NaN,NaN,...,NaN,OH,PO Box 626,NaN,NaN,NaN,43764,7,Po Box 626,220840
59759,701176,Northern Local SD,3,12,2011,1993-02-01,new lexington,NaN,NaN,NaN,...,NaN,OH,813 Johnson Ave,NaN,NaN,NaN,43764,7,813 Johnson Ave,251956
59761,701178,Northern Local SD,3,12,2011,1993-09-01,pleasantville,NaN,NaN,NaN,...,NaN,OH,4355 East Main St,NaN,NaN,NaN,43148,7,4355 E Main St,161585
59762,701179,Northern Local SD,3,12,2011,1993-08-01,glenford,NaN,NaN,NaN,...,NaN,OH,9980 Brownsville Rd,NaN,NaN,NaN,43739,7,9980 Brownsville Rd,161582


In [21]:
withdraw_na.groupby(['district','school_year']).agg({'student_lookup':'count'})

student_lookup
district              school_year                
Logan_Hocking         2011                    282
                      2012                     16
                      2013                      4
                      2014                      3
                      2015                      3
Morgan                2012                      1
                      2013                      4
                      2014                      5
                      2015                    159
New Lexington City SD 2011                    110
                      2012                      8
                      2013                      2
                      2014                     15
                      2015                     18
Northern Local SD     2011                    161
                      2012                      1
                      2013                      2
                      2014                      9
                      2015                    168
Zanesville            2011                    263
                      2012                    232
                      2013                    232
                      2014                    240
                      2015                    191

In [22]:
all_students.join(withdraw_na.groupby(['district','school_year']).agg({'student_lookup':'count'}), rsuffix='_na')

student_lookup  student_lookup_na
district   school_year                                   
Coshocton  2009                    161                NaN
           2010                    161                NaN
           2011                    143                NaN
           2012                    126                NaN
           2013                    108                NaN
...                                ...                ...
Zanesville 2011                    263              263.0
           2012                    232              232.0
           2013                    232              232.0
           2014                    240              240.0
           2015                    191              191.0

[85 rows x 2 columns]

# Certain districts in certain years seem to not use `withdraw_reason` for grade 12 

## Zanesville simply lacks data outside of 2015, but other years where there is effectively total missingness on withdraw reason appear to have `graduation_date` listed for most students, and can likely be used as an imputation.

In [23]:
missing_by_district = all_students.join(withdraw_na.groupby(['district','school_year']).agg({'student_lookup':'count'}), rsuffix='_na')

missing_by_district[missing_by_district['student_lookup_na'].notnull()]

student_lookup  student_lookup_na
district              school_year                                   
Logan_Hocking         2011                    282              282.0
                      2012                    326               16.0
                      2013                    333                4.0
                      2014                    304                3.0
                      2015                    330                3.0
Morgan                2012                    153                1.0
                      2013                    167                4.0
                      2014                    135                5.0
                      2015                    174              159.0
New Lexington City SD 2011                    110              110.0
                      2012                    151                8.0
                      2013                    153                2.0
                      2014                    147               15.0
                      2015                    124               18.0
Northern Local SD     2011                    162              161.0
                      2012                    172                1.0
                      2013                    150                2.0
                      2014                    196                9.0
                      2015                    194              168.0
Zanesville            2011                    263              263.0
                      2012                    232              232.0
                      2013                    232              232.0
                      2014                    240              240.0
                      2015                    191              191.0

# Raw data for withdraw reason for students in 12th grade; also includes any student with a `graduate` withdraw reason since it is possible to graduate early

## use DataFrame `grad_df`

In [4]:
qry = '''
SELECT student_lookup,
    grade,
    school_year,
    withdraw_reason
from clean.all_snapshots
where grade = 12 or withdraw_reason = 'graduate'
order by student_lookup;
'''

cur.execute(qry)

rows = cur.fetchall()

# Build dataframe from rows
grad_df = pd.DataFrame(rows, columns=[name[0] for name in cur.description])

# Make sure student_id is an int
grad_df['student_lookup'] = grad_df['student_lookup'].astype('int')

grad_df.head()

,student_lookup,grade,school_year,withdraw_reason
0,46,12.0,2015,dropout - over 18
1,46,12.0,2014,did not withdraw
2,47,12.0,2015,transferred - in state
3,49,12.0,2015,graduate
4,50,12.0,2015,graduate


In [5]:
len(grad_df)

20096

In [6]:
grad_df['withdraw_reason'].replace(to_replace=[None], value='Missing', inplace=True)

In [7]:
cnt_withdraw = grad_df.groupby(['school_year','withdraw_reason']).agg({'student_lookup':'count'})

In [8]:
# Withdraw reasons for 12th graders by year
cnt_withdraw.unstack(0).replace(np.nan, 0).astype('int')

student_lookup                                \
school_year                              2006  2007  2008  2009  2010  2011   
withdraw_reason                                                               
Missing                                  1693  1393     0     0     0   816   
did not withdraw                            0     0    39    28    49    54   
dropout - attendance                        0     0     2     0     1     3   
dropout - did not finish tests              0     0     0     0     0     0   
dropout - employment                        0     0     0     0     0     0   
dropout - moved                             0     0     2     0     0     1   
dropout - over 18                           0     0    26    15    16    21   
error                                       0     0     0     0     0     0   
expelled                                    0     0     1     0     1     1   
graduate                                    0     0  1135  1086  1281  1255   
transferred - court ordered                 0     0     1     0     0     2   
transferred - homeschool                    0     0     0     1     1     1   
transferred - in state                      0     0   165   184   164   222   
transferred - out of country                0     0    12     9     6     5   
transferred - out of state                  0     0    13     8    11    11   
transferred - private                       0     0     2     0     4     1   
withdrew - death                            0     0     0     0     0     0   
withdrew - illness                          0     0     0     0     2     0   

                                                        
school_year                     2012  2013  2014  2015  
withdraw_reason                                         
Missing                          258   244   272   539  
did not withdraw                  32    68   198   309  
dropout - attendance               9     6    14     7  
dropout - did not finish tests    11    15     5     5  
dropout - employment               0     0     1     2  
dropout - moved                    3     3     4     1  
dropout - over 18                 43    20    42    44  
error                              0     0     1    11  
expelled                           6     0     0     0  
graduate                        1984  1837  1911  1429  
transferred - court ordered        1     1     1     0  
transferred - homeschool           4     0     2     2  
transferred - in state           265   253   185   211  
transferred - out of country       7    11     1     2  
transferred - out of state        16     9    16    19  
transferred - private              1     3     1     3  
withdrew - death                   1     1     0     2  
withdrew - illness                 0     0     0     0

# All (student, school_year) entering grade 10

In [15]:
# Gets all students entering grade 10 at school year
qry = '''
SELECT distinct student_lookup,
    grade,
    school_year
from clean.all_snapshots
where grade = 10
order by student_lookup;
'''

cur.execute(qry)

rows = cur.fetchall()

# Build dataframe from rows
df = pd.DataFrame(rows, columns=[name[0] for name in cur.description])

# Make sure student_id is an int
df['student_lookup'] = df['student_lookup'].astype('int')

df.head()

,student_lookup,grade,school_year
0,45,10,2014
1,46,10,2012
2,47,10,2013
3,48,10,2012
4,49,10,2013


# Links the future "withdraw reason" in grade 12 to the student entering 10th grade

## Use DataFrame `grd_10`

In [16]:

# Left join means it keeps all 10th grade students, even if they didn't appear in the grad_df
grd_10 = pd.merge(df, grad_df, how='left', on='student_lookup')
grd_10

,student_lookup,grade_x,school_year_x,grade_y,school_year_y,withdraw_reason
0,45,10,2014,NaN,NaN,NaN
1,46,10,2012,12.0,2015.0,dropout - over 18
2,46,10,2012,12.0,2014.0,did not withdraw
3,47,10,2013,12.0,2015.0,transferred - in state
4,48,10,2012,NaN,NaN,NaN
...,...,...,...,...,...,...
20373,701167,10,2011,NaN,NaN,NaN
20374,701170,10,2011,NaN,NaN,NaN
20375,701177,10,2011,NaN,NaN,NaN
20376,701180,10,2011,NaN,NaN,NaN


In [18]:
grd_10.columns = ['student_lookup',	'grade_10',	'yr_grade_10',	'grade_12',	'yr_grade_12',	'grade_12_withdraw']

In [20]:
grd_10

,student_lookup,grade_10,yr_grade_10,grade_12,yr_grade_12,grade_12_withdraw
0,45,10,2014,NaN,NaN,NaN
1,46,10,2012,12.0,2015.0,dropout - over 18
2,46,10,2012,12.0,2014.0,did not withdraw
3,47,10,2013,12.0,2015.0,transferred - in state
4,48,10,2012,NaN,NaN,NaN
...,...,...,...,...,...,...
20373,701167,10,2011,NaN,NaN,NaN
20374,701170,10,2011,NaN,NaN,NaN
20375,701177,10,2011,NaN,NaN,NaN
20376,701180,10,2011,NaN,NaN,NaN


In [25]:
grd_10.groupby(['yr_grade_10', 'grade_12_withdraw']).agg({'student_lookup':'count'}).unstack(0).replace(np.nan, 0).astype('int')

student_lookup                               \
yr_grade_10                              2006  2007  2008 2009  2010  2011   
grade_12_withdraw                                                            
Missing                                     2     0     3   23    16   236   
did not withdraw                           29    23    31   30    25    53   
dropout - attendance                        2     0     1    1     1     6   
dropout - did not finish tests              0     0     0    0     1     9   
dropout - employment                        0     0     0    0     0     0   
dropout - moved                             1     0     0    1     3     1   
dropout - over 18                          11    13    19   17    13    18   
error                                       0     0     0    0     0     0   
expelled                                    1     0     1    0     2     0   
graduate                                  974  1046  1029  979  1222  1619   
transferred - court ordered                 1     0     0    0     0     1   
transferred - homeschool                    0     1     1    1     2     0   
transferred - in state                     97   131   111  129   154   200   
transferred - out of country                0     0     0    0     0     1   
transferred - out of state                 10     2     7    5     8     4   
transferred - private                       1     0     3    1     1     3   
withdrew - death                            0     0     0    0     1     1   
withdrew - illness                          0     0     2    0     0     0   

                                                 
yr_grade_10                     2012  2013 2014  
grade_12_withdraw                                
Missing                          249   514    6  
did not withdraw                  40   437    0  
dropout - attendance              12     4    0  
dropout - did not finish tests     4     4    0  
dropout - employment               0     1    0  
dropout - moved                    3     1    0  
dropout - over 18                 45    17    0  
error                              5     4    1  
expelled                           0     0    0  
graduate                        1802  1261   14  
transferred - court ordered        0     0    0  
transferred - homeschool           1     2    0  
transferred - in state           148   164    1  
transferred - out of country       0     0    0  
transferred - out of state         9     8    0  
transferred - private              2     1    1  
withdrew - death                   0     2    0  
withdrew - illness                 0     0    0

# Data obtained by the View (sketch.hs_withdraw_info) WITHOUT further deduplication (grade 10)

## Use DataFrame `hs_w`

In [37]:
qry = '''
SELECT * from sketch.hs_withdraw_info
WHERE grade=10 and entry_year BETWEEN 2007 AND 2013;
'''

cur.execute(qry)

rows = cur.fetchall()

# Build dataframe from rows
hs_w = pd.DataFrame(rows, columns=[name[0] for name in cur.description])

# Make sure student_id is an int
hs_w ['student_lookup'] = hs_w['student_lookup'].astype('int')

hs_w[:10]

,student_lookup,grade,entry_year,grad_year,grad_year_missing,dropout_year,dropout_year_missing,in_state_transfer_year,in_state_transfer_year_missing,transfer_out_year,transfer_out_year_missing,withdrew_year,withdrew_year_missing,all_missing
0,46,10,2012,NaN,1,2015.0,0,NaN,1,NaN,1,NaN,1,0
1,47,10,2013,NaN,1,NaN,1,2015.0,0,NaN,1,NaN,1,0
2,48,10,2012,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1
3,49,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0
4,50,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0
5,269,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0
6,286,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0
7,297,10,2012,NaN,1,2015.0,0,NaN,1,NaN,1,NaN,1,0
8,338,10,2013,NaN,1,NaN,1,2012.0,0,NaN,1,NaN,1,0
9,407,10,2013,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1


In [41]:
grd_10[grd_10['student_lookup']==47]

,student_lookup,grade_10,yr_grade_10,grade_12,yr_grade_12,grade_12_withdraw
3,47,10,2013,12.0,2015.0,transferred - in state


In [36]:
# grd_10 students entering 2007-2013 (our cohorts of interest)
len(grd_10[grd_10['yr_grade_10'].isin(list(range(2007,2014)))])

13894

# Current data retrieval

In [26]:
cur.execute('''
                select *
                from (
		               SELECT *, ROW_NUMBER() OVER
		                    (PARTITION BY student_lookup, grade
                            ORDER BY student_lookup) AS rnum
		               FROM sketch.hs_withdraw_info hwi) t
                where t.rnum = 1
                and t.grade = 10
                and t.entry_year >= 2007 and t.entry_year <= 2013
                and ((t.grad_year is not null or t.dropout_year is not null)
                		or (t.transfer_out_year is null))
                and ((t.grad_year is not null or t.dropout_year is not null)
               			or (t.in_state_transfer_year is null));
                ''')

rows = cur.fetchall()

# Build dataframe from rows
existing = pd.DataFrame(rows, columns=[name[0] for name in cur.description])

# Make sure student_id is an int
existing['student_lookup'] = existing['student_lookup'].astype('int')

existing

,student_lookup,grade,entry_year,grad_year,grad_year_missing,dropout_year,dropout_year_missing,in_state_transfer_year,in_state_transfer_year_missing,transfer_out_year,transfer_out_year_missing,withdrew_year,withdrew_year_missing,all_missing,rnum
0,46,10,2012,NaN,1,2015.0,0,NaN,1,NaN,1,NaN,1,0,1
1,48,10,2012,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1,1
2,49,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0,1
3,50,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0,1
4,269,10,2013,2015.0,0,NaN,1,NaN,1,NaN,1,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10847,701138,10,2011,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1,1
10848,701170,10,2011,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1,1
10849,701177,10,2011,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1,1
10850,701180,10,2011,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,1,1


13894

In [28]:
list(range(2007,2014))

[2007, 2008, 2009, 2010, 2011, 2012, 2013]